### Conversational AI - aka Chatbot!

In [5]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr

In [6]:
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

DeepSeek API Key exists and begins sk-


In [7]:
deepseek_url = "https://api.deepseek.com"

deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)

In [10]:
result = deepseek.chat.completions.create(model='deepseek-chat', messages=[{"role": "user", "content": "Hi, how are you?"}])
display(Markdown(result.choices[0].message.content))

Hi there! 😊 I'm doing great, thank you for asking! I'm really excited to be here and ready to help you with whatever you need. How are you doing today? Is there anything interesting you'd like to talk about or any questions I can help answer?

In [11]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

#### And now, writing a new callback
We now need to write a function called:

chat(message, history)

Which will be a callback function we will give gradio.

#### The job of this function
Take a message, take the prior conversation, and return the response.

In [12]:
def chat(message, history):
    return "bananas"

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


In [14]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


### Let's update the chat method

In [16]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = deepseek.chat.completions.create(model='deepseek-chat', messages=messages)
    return response.choices[0].message.content

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Update the chat to do streaming!

In [18]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    stream = deepseek.chat.completions.create(model='deepseek-chat', messages=messages, stream=True)

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


Update the system message

In [20]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."


In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


In [22]:
system_message += "\n if the customer asks for shoes, you have to let them know that shoes are not on sale. \
    you should try to gently encourage the customer to try items that are on sale."

In [23]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


In [ ]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    updated_system_message = system_message
    if 'belt' in message.lower() or 'belts' in message.lower():
        updated_system_message += "\n The store does not sell belts, you should try to gently encourage the customer to try other items"

    messages = [{'role': 'system', 'content': updated_system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    stream = deepseek.chat.completions.create(model='deepseek-chat', messages=messages, stream=True)

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [29]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.
